In [1]:
pip install tabulate emoji textstat

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install transformers[torch] accelerate textstat

   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
    --------------------------------------- 0.3/12.0 MB ? eta -:--:--
   --- ------------------------------------ 1.0/12.0 MB 2.8 MB/s eta 0:00:04
   ----- ---------------------------------- 1.6/12.0 MB 3.0 MB/s eta 0:00:04
   ------- -------------------------------- 2.4/12.0 MB 3.1 MB/s eta 0:00:04
   --------- ------------------------------ 2.9/12.0 MB 3.1 MB/s eta 0:00:03
   ------------- -------------------------- 3.9/12.0 MB 3.3 MB/s eta 0:00:03
   --------------- ------------------------ 4.7/12.0 MB 3.4 MB/s eta 0:00:03
   ------------------ --------------------- 5.5/12.0 MB 3.4 MB/s eta 0:00:02
   -------------------- ------------------- 6.3/12.0 MB 3.5 MB/s eta 0:00:02
   ----------------------- ---------------- 7.1/12.0 MB 3.5 MB/s eta 0:00:02
   -------------------------- ------------- 7.9/12.0 MB 3.6 MB/s eta 0:00:02
   --------------------------- ------------ 8.4/12.0 MB 3.5 MB/s eta 0:00:02
   ----------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [4]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [7]:
import torch

OSError: [WinError 1114] Routine di inizializzazione della libreria di collegamento dinamico (DLL) non riuscita. Error loading "c:\Users\Rosy\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\lib\c10.dll" or one of its dependencies.

In [8]:

import joblib
import numpy as np
import textstat
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Carica i componenti salvati (assicurati che i percorsi siano corretti)
SCALER_PATH = 'pkl/scaler.pkl'
MODEL_PATH = 'bert_ai_detector/' # La cartella salvata nel notebook ibrido
RF_MODEL_PATH = 'pkl/rf_hybrid_model.pkl' # Se hai usato RF come testa finale

scaler = joblib.load(SCALER_PATH)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
bert_model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH)
bert_model.eval()

OSError: [WinError 1114] Routine di inizializzazione della libreria di collegamento dinamico (DLL) non riuscita. Error loading "c:\Users\Rosy\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\lib\c10.dll" or one of its dependencies.

In [ ]:
def extract_test_signatures(text):
    # Calcolo delle 13 feature basate sul tuo notebook 'signature.ipynb'
    stats = {
        'avg_word_length': np.mean([len(w) for w in text.split()]) if text.split() else 0,
        'sentence_count': textstat.sentence_count(text),
        'flesch_reading_ease': textstat.flesch_reading_ease(text),
        'lexical_diversity': len(set(text.split())) / len(text.split()) if text.split() else 0,
        'stopword_ratio': 0.15, # Esempio: qui dovresti inserire la logica di conteggio stopword
        # ... aggiungi le altre feature che hai definito nel notebook signature
    }
    # Converte in array e scala
    features = np.array(list(stats.values())).reshape(1, -1)
    return scaler.transform(features)

In [ ]:
def predict_ai_vs_human(text):
    # 1. BERT Output
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = bert_model(**inputs)
        bert_probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        bert_score = bert_probs[0][1].item()  # Probabilità classe AI

    # 2. Stylometric Signatures
    sig_features = extract_test_signatures(text)

    # 3. Decisione Finale (Logica Ibrida)
    # Se hai un modello Random Forest finale:
    # final_input = np.hstack([sig_features, [[bert_score]]])
    # final_pred = rf_model.predict(final_input)
    
    # Esempio di output semplice:
    print(f"BERT Confidence (AI): {bert_score:.2%}")
    return "AI" if bert_score > 0.5 else "HUMAN"

# TEST PRATICO
testo_da_testare = "Il tuo testo qui..."
risultato = predict_ai_vs_human(testo_da_testare)
print(f"Risultato: {risultato}")